# Análisis Exploratorio de Datos - Clientes Bancarios
## Este notebook realiza el análisis exploratorio de datos de un dataset de clientes bancarios con Python

## 1. Importación de librerías y carga de datos

In [26]:
# Importar librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Configuración de visualización
plt.style.use('default')
sns.set_theme()
%matplotlib inline

# Importar funciones de carga de datos
import sys
sys.path.append('../src')
from data.data_loader import load_customer_data, load_bank_data, get_basic_info

In [27]:
# Cargar datos de clientes
df_customers = load_customer_data()

# Cargar datos bancarios
df_bank = load_bank_data()

In [28]:
# Información básica de los datos de clientes
print("=== Información de Datos de Clientes ===")
get_basic_info(df_customers)

# Información básica de los datos bancarios
print("\n=== Información de Datos Bancarios ===")
get_basic_info(df_bank)

=== Información de Datos de Clientes ===

Información del DataFrame:
<class 'pandas.core.frame.DataFrame'>
Index: 20115 entries, 0 to 20114
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Income             20115 non-null  int64         
 1   Kidhome            20115 non-null  int64         
 2   Teenhome           20115 non-null  int64         
 3   Dt_Customer        20115 non-null  datetime64[ns]
 4   NumWebVisitsMonth  20115 non-null  int64         
 5   ID                 20115 non-null  object        
dtypes: datetime64[ns](1), int64(4), object(1)
memory usage: 1.1+ MB
None

Primeras 5 filas:
   Income  Kidhome  Teenhome Dt_Customer  NumWebVisitsMonth  \
0  161770        1         0  2012-04-04                 29   
1   85477        1         1  2012-12-30                  7   
2  147233        1         1  2012-02-02                  5   
3  121393        1         2  2012-12-21   

## 2. Transformación y limpieza de los datos

### 2.1 Estandarización de nombres de columnas

In [29]:
# Mostrar columnas de cada dataset
print("Columnas en datos de clientes:")
print(df_customers.columns.tolist())

print("\nColumnas en datos bancarios:")
print(df_bank.columns.tolist())

Columnas en datos de clientes:
['Income', 'Kidhome', 'Teenhome', 'Dt_Customer', 'NumWebVisitsMonth', 'ID']

Columnas en datos bancarios:
['age', 'job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed', 'y', 'date', 'latitude', 'longitude', 'id_']


In [30]:
# Rename de las columnas para una mejor comprensión de su significado, ya que algunas columnas presentan un exceso de abreviación
# Renombrar columnas del dataset de clientes
customer_columns_rename = {
    'Income': 'Income',
    'Kidhome': 'Number_of_Kids',
    'Teenhome': 'Number_of_Teenagers',
    'Dt_Customer': 'Registration_Date',
    'NumWebVisitsMonth': 'Monthly_Web_Visits',
    'ID': 'Customer_ID'
}

# Renombrar columnas del dataset bancario
bank_columns_rename = {
    'age': 'Age',
    'job': 'Job',
    'marital': 'Marital_Status',
    'education': 'Education_Level',
    'default': 'Credit_Default',
    'housing': 'Mortgage_Loan',
    'loan': 'Personal_Loan',
    'contact': 'Contact_Type',
    'duration': 'Call_Duration',
    'campaign': 'Campaign_Contacts',
    'pdays': 'Days_Since_Last_Contact',
    'previous': 'Previous_Contacts',
    'poutcome': 'Previous_Campaign_Outcome',
    'emp.var.rate': 'Employment_Variation_Rate',
    'cons.price.idx': 'Consumer_Price_Index',
    'cons.conf.idx': 'Consumer_Confidence_Index',
    'euribor3m': 'Euribor_3M_Rate',
    'nr.employed': 'Number_of_Employees',
    'y': 'Subscribed_to_Service',
    'date': 'Date',
    'latitude': 'Latitude',
    'longitude': 'Longitude',
    'id_': 'Customer_ID'
}

# Aplicar los cambios
df_customers = df_customers.rename(columns=customer_columns_rename)
df_bank = df_bank.rename(columns=bank_columns_rename)

# Verificar los cambios
print("Columnas en datos de clientes después del rename:")
print(df_customers.columns.tolist())
print("\nColumnas en datos bancarios después del rename:")
print(df_bank.columns.tolist())

Columnas en datos de clientes después del rename:
['Income', 'Number_of_Kids', 'Number_of_Teenagers', 'Registration_Date', 'Monthly_Web_Visits', 'Customer_ID']

Columnas en datos bancarios después del rename:
['Age', 'Job', 'Marital_Status', 'Education_Level', 'Credit_Default', 'Mortgage_Loan', 'Personal_Loan', 'Contact_Type', 'Call_Duration', 'Campaign_Contacts', 'Days_Since_Last_Contact', 'Previous_Contacts', 'Previous_Campaign_Outcome', 'Employment_Variation_Rate', 'Consumer_Price_Index', 'Consumer_Confidence_Index', 'Euribor_3M_Rate', 'Number_of_Employees', 'Subscribed_to_Service', 'Date', 'Latitude', 'Longitude', 'Customer_ID']


### 2.2 Corrección de formato y transformación de columnas

In [31]:
# En el dataset de clientes:

# Convertir Registration_Date a formato dd/mm/yyyy
df_customers['Registration_Date'] = pd.to_datetime(df_customers['Registration_Date']).dt.strftime('%d/%m/%Y')



In [32]:
# En el dataset de datos bancarios:

# Pasar los valores de Age de float a integer, ya que la edad no puede ser un número decimal.
df_bank['Age'] = df_bank['Age'].apply(lambda x: int(x) if pd.notnull(x) else x)

# Detectar columnas numéricas que usan coma decimal
columnas_numericas = ['Consumer_Price_Index', 'Consumer_Confidence_Index', 'Employment_Variation_Rate', 'Euribor_3M_Rate']

# Reemplazar comas por puntos en las columnas numéricas y convertir a float
for col in columnas_numericas:
    # Verificar si hay comas en los valores numericos y sustituir por punto decimal
    if df_bank[col].dtype == 'object' and df_bank[col].str.contains(',').any():
        df_bank[col] = df_bank[col].str.replace(',', '.').astype('float64')

# Verificar si Credit_Default tiene valores distintos a 0.0 o nulo. Si no, eliminar la columna ya que no aporta valor para el análisis.
valores_unicos = df_bank['Credit_Default'].unique()
valores_no_nulos = [x for x in valores_unicos if pd.notnull(x) and x != 0.0]

if len(valores_no_nulos) == 0:
    # Si solo tiene valores 0.0 o nulos, eliminar la columna
    df_bank = df_bank.drop('Credit_Default', axis=1)
    print("La columna Credit_Default ha sido eliminada")
else:
    print("La columna Credit_Default contiene los siguientes valores distintos a 0.0:", valores_no_nulos)

# Pasar Mortgage_Loan y Personal_Loan a booleanos, en lugar de usar 1.0 y 0.0.
df_bank['Mortgage_Loan'] = df_bank['Mortgage_Loan'].map({1.0: True, 0.0: False})
df_bank['Personal_Loan'] = df_bank['Personal_Loan'].map({1.0: True, 0.0: False})

# Convertir fechas en formato 'dd-mes-yyyy' a 'dd/mm/yyyy'
meses_num = {
    'enero': '01', 'febrero': '02', 'marzo': '03', 'abril': '04',
    'mayo': '05', 'junio': '06', 'julio': '07', 'agosto': '08',
    'septiembre': '09', 'octubre': '10', 'noviembre': '11', 'diciembre': '12'
}

def convertir_fecha(fecha):
    if pd.isna(fecha):
        return fecha
    dia, mes, anyo = fecha.split('-')
    mes = meses_num[mes.lower()]
    return f"{dia.zfill(2)}/{mes}/{anyo}"   #zfill(2) asegura que el día tenga 2 dígitos añadiendo un 0 a la izq en caso de que sea menor a 10

df_bank['Date'] = df_bank['Date'].apply(convertir_fecha)


# Latitude y Longitude tienen algunos valores decimales y otros string, por como se muestran en la tabla, asi que se convierten a float ambas columnas.
df_bank['Latitude'] = df_bank['Latitude'].astype(float)
df_bank['Longitude'] = df_bank['Longitude'].astype(float)

# Agrupar Latitud y Longitud en una columna de Coordinates. 
df_bank['Coordinates'] = df_bank['Latitude'].astype(str) + ', ' + df_bank['Longitude'].astype(str)

# Categorizar la columna Days_Since_Last_Contact en "Contacted" y "Not contacted" dependiendo si el valor es menor de 999.
df_bank['Days_Since_Last_Contact'] = df_bank['Days_Since_Last_Contact'].apply(
    lambda x: 'Contacted' if x < 999 
    else 'Not contacted'
)

# Categorizar la columna Call_Duration en "Very short", "Short", "Medium", "Long" y "Very long" dependiendo del valor.
df_bank['Call_Duration'] = df_bank['Call_Duration'].apply(
    lambda x: 'Very short' if x < 10 
    else 'Short' if x < 20 
    else 'Medium' if x < 30 
    else 'Long' if x < 40 
    else 'Very long'
)


La columna Credit_Default contiene los siguientes valores distintos a 0.0: [np.float64(1.0)]


### 2.3 Análisis de valores nulos

In [33]:
# Análisis de valores nulos en datos de clientes
print("=== Valores Nulos en Datos de Clientes ===")
null_customers = df_customers.isnull().sum()
print(null_customers[null_customers > 0])

# Análisis de valores nulos en datos bancarios
print("\n=== Valores Nulos en Datos Bancarios ===")
null_bank = df_bank.isnull().sum()
print(null_bank[null_bank > 0])

=== Valores Nulos en Datos de Clientes ===
Series([], dtype: int64)

=== Valores Nulos en Datos Bancarios ===
Age                     5120
Job                      345
Marital_Status            85
Education_Level         1807
Credit_Default          8981
Mortgage_Loan           1026
Personal_Loan           1026
Consumer_Price_Index     471
Euribor_3M_Rate         9256
Date                     248
dtype: int64


### 2.4 Tratado de los nulos

In [34]:
# Clasificar tipos de columnas
categorical_cols = ['Job', 'Marital_Status', 'Education_Level', 'Credit_Default', 'Mortgage_Loan', 'Personal_Loan']
numeric_cols = ['Age', 'Consumer_Price_Index', 'Euribor_3M_Rate']
date_cols = ['Date']

# === Reemplazo de nulos en columnas categóricas con "Unknown"
df_bank[categorical_cols] = df_bank[categorical_cols].fillna("Unknown")

# === Reemplazo de nulos en columnas numéricas con la media de la columna
for col in numeric_cols:
    mean_value = df_bank[col].mean()
    df_bank[col] = df_bank[col].fillna(mean_value)

# === Reemplazo de nulos en columnas de fecha con la media de la fecha
# Asegurar que sea datetime, errors='coerce' convierte strings invalidos a NaT (NaN para fechas)
df_bank['Date'] = pd.to_datetime(df_bank['Date'], errors='coerce')

# Calcular la media temporal (convertir a timestamp -> sacar media -> volver a datetime)
mean_timestamp = df_bank['Date'].dropna().astype(np.int64).mean()
mean_date = pd.to_datetime(mean_timestamp)

# Reemplazar los nulos por la fecha promedio
df_bank['Date'] = df_bank['Date'].fillna(mean_date)


# Verifica que no queden nulos
print(df_bank.isnull().sum().sort_values(ascending=False))

Age                          0
Job                          0
Marital_Status               0
Education_Level              0
Credit_Default               0
Mortgage_Loan                0
Personal_Loan                0
Contact_Type                 0
Call_Duration                0
Campaign_Contacts            0
Days_Since_Last_Contact      0
Previous_Contacts            0
Previous_Campaign_Outcome    0
Employment_Variation_Rate    0
Consumer_Price_Index         0
Consumer_Confidence_Index    0
Euribor_3M_Rate              0
Number_of_Employees          0
Subscribed_to_Service        0
Date                         0
Latitude                     0
Longitude                    0
Customer_ID                  0
Coordinates                  0
dtype: int64


In [35]:
# Información básica de los datos de clientes
print("=== Información de Datos de Clientes ===")
get_basic_info(df_customers)

# Información básica de los datos bancarios
print("\n=== Información de Datos Bancarios ===")
get_basic_info(df_bank)

=== Información de Datos de Clientes ===

Información del DataFrame:
<class 'pandas.core.frame.DataFrame'>
Index: 20115 entries, 0 to 20114
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Income               20115 non-null  int64 
 1   Number_of_Kids       20115 non-null  int64 
 2   Number_of_Teenagers  20115 non-null  int64 
 3   Registration_Date    20115 non-null  object
 4   Monthly_Web_Visits   20115 non-null  int64 
 5   Customer_ID          20115 non-null  object
dtypes: int64(4), object(2)
memory usage: 1.1+ MB
None

Primeras 5 filas:
   Income  Number_of_Kids  Number_of_Teenagers Registration_Date  \
0  161770               1                    0        04/04/2012   
1   85477               1                    1        30/12/2012   
2  147233               1                    1        02/02/2012   
3  121393               1                    2        21/12/2012   
4   63164             

### 2.5 Eliminación de duplicados

In [36]:
# Detectar si hay filas duplicadas en el dataset de datos bancarios
banco_duplicados = df_bank[df_bank.duplicated()]
print(banco_duplicados)

# Y en el dataset de clientes
clientes_duplicados = df_customers[df_customers.duplicated()]
print(clientes_duplicados)



Empty DataFrame
Columns: [Age, Job, Marital_Status, Education_Level, Credit_Default, Mortgage_Loan, Personal_Loan, Contact_Type, Call_Duration, Campaign_Contacts, Days_Since_Last_Contact, Previous_Contacts, Previous_Campaign_Outcome, Employment_Variation_Rate, Consumer_Price_Index, Consumer_Confidence_Index, Euribor_3M_Rate, Number_of_Employees, Subscribed_to_Service, Date, Latitude, Longitude, Customer_ID, Coordinates]
Index: []

[0 rows x 24 columns]
Empty DataFrame
Columns: [Income, Number_of_Kids, Number_of_Teenagers, Registration_Date, Monthly_Web_Visits, Customer_ID]
Index: []
